In [6]:
import os
import json
import re
import pandas as pd

def parse_jsonl(data):
    parsed_data = []
    for line in data:
        parsed_data.append(json.loads(line))

    return parsed_data

def get_main_df(path, file_in_question):

    main_df = pd.DataFrame()
    for root, dirs, files in os.walk(path):
        if "layer" not in root:
            continue

        for file in files:

            if file == file_in_question:
                with open(os.path.join(root, file)) as f:
                    data = f.readlines()


                parsed_data = parse_jsonl(data)
                df = pd.DataFrame(parsed_data)
                
                # We have tp, fp, tn, fn
                # We want precision, recall, f1
                df["precision"] = df["tp"] / (df["tp"] + df["fp"])
                df["recall"] = df["tp"] / (df["tp"] + df["fn"])
                df["f1"] = 2 * (df["precision"] * df["recall"]) / (df["precision"] + df["recall"])
                df["accuracy"] = (df["tp"] + df["tn"]) / (df["tp"] + df["tn"] + df["fp"] + df["fn"])
                df["layer"] = re.search(r"layer(\d+)", root).group(1)

                main_df = pd.concat([main_df, df])

    return main_df

def get_best(main_df, specific_layer:int=None):
    if specific_layer is not None:
        print(f"Filtering for layer {specific_layer}")
        main_df = main_df[main_df["layer"] == str(specific_layer)]
    best_accuracy = main_df[main_df["accuracy"] == main_df["accuracy"].max()]["accuracy"]
    best_f1 = main_df[main_df["f1"] == main_df["f1"].max()]["f1"]
    layer = main_df[main_df["f1"] == main_df["f1"].max()]["layer"]
    print(f"Best layer: {layer.values[0]}")
    sorted_df = main_df.sort_values(by=["f1"], ascending=False)
    return best_accuracy, best_f1, sorted_df


path = "gpt2_gbug-java"
file_in_question = "random_forest.jsonl"
main_df = get_main_df(path, file_in_question)
best_accuracy, best_f1, best_restult = get_best(main_df, 
                                                specific_layer=0)
print(f"For {file_in_question} in {path}")
print(f"BEST F1: {best_f1}")
print(f"BEST ACCURACY: {best_accuracy}")
best_restult

Filtering for layer 0
Best layer: 0
For random_forest.jsonl in gpt2_gbug-java
BEST F1: 0    0.682927
Name: f1, dtype: float64
BEST ACCURACY: 24    0.7
Name: accuracy, dtype: float64


,classifier,n_features,tp,fp,tn,fn,y_pred,y_test,best_params,dataset,precision,recall,f1,accuracy,layer
0,random_forest,1,28,24,6,2,"[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': None, 'max_features': 'sqrt', 'm...",gpt2_gbug-java/layer0/,0.538462,0.933333,0.682927,0.566667,0
26,random_forest,200,23,15,15,7,"[1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': 50, 'max_features': 'sqrt', 'min...",gpt2_gbug-java/layer0/,0.605263,0.766667,0.676471,0.633333,0
1,random_forest,2,26,21,9,4,"[1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': None, 'max_features': 'sqrt', 'm...",gpt2_gbug-java/layer0/,0.553191,0.866667,0.675325,0.583333,0
17,random_forest,2,26,21,9,4,"[1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': 50, 'max_features': 'sqrt', 'min...",gpt2_gbug-java/layer0/,0.553191,0.866667,0.675325,0.583333,0
16,random_forest,1,27,23,7,3,"[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': 50, 'max_features': 'sqrt', 'min...",gpt2_gbug-java/layer0/,0.540000,0.900000,0.675000,0.566667,0
2,random_forest,3,25,20,10,5,"[1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': None, 'max_features': 'sqrt', 'm...",gpt2_gbug-java/layer0/,0.555556,0.833333,0.666667,0.583333,0
3,random_forest,4,25,20,10,5,"[1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': None, 'max_features': 'sqrt', 'm...",gpt2_gbug-java/layer0/,0.555556,0.833333,0.666667,0.583333,0
18,random_forest,3,25,20,10,5,"[1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': None, 'max_features': 'sqrt', 'm...",gpt2_gbug-java/layer0/,0.555556,0.833333,0.666667,0.583333,0
12,random_forest,1000,22,14,16,8,"[1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': 50, 'max_features': 'sqrt', 'min...",gpt2_gbug-java/layer0/,0.611111,0.733333,0.666667,0.633333,0
24,random_forest,50,18,6,24,12,"[0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","{'max_depth': 50, 'max_features': 'sqrt', 'min...",gpt2_gbug-java/layer0/,0.750000,0.600000,0.666667,0.700000,0


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style for better aesthetics
sns.set_style("whitegrid")

from dataclasses import dataclass

@dataclass
class Metric:
    name: str
    label: str

F1 = Metric("f1", "F1 Score")
ACCURACY = Metric("accuracy", "Accuracy")

ACTIVE_METRIC = F1

# Create the figure
plt.figure(figsize=(8, 6))

# Plot mean F1 score across all layers with confidence interval
sns.lineplot(data=main_df, x="n_features", y=ACTIVE_METRIC.name, ci=95, label=f"Mean {ACTIVE_METRIC.label}", color="black", linestyle="dashed")

# Plot Layer 11
# df_11 = main_df[main_df["layer"] == "11"]
# plt.plot(df_11["n_features"], df_11[ACTIVE_METRIC.name], label="Layer 11", color="red", linestyle="-",alpha=0.3)

# # Plot Layer 1
# df_1 = main_df[main_df["layer"] == "1"]
# plt.plot(df_1["n_features"], df_1[ACTIVE_METRIC.name], label="Layer 1", color="blue", linestyle="-",alpha=0.3)

# Labels and title
plt.xlabel("Number of Features", fontsize=12)
plt.ylabel(ACTIVE_METRIC.label, fontsize=12)
plt.title(f"{ACTIVE_METRIC.label} with Confidence Interval Across Layers", fontsize=14, fontweight="bold")

# Add a legend
plt.legend(fontsize=10)

# make the x log scale
plt.xscale("log")

# Improve layout
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()

#define axis limits
plt.ylim(0, 1)

# Show the plot
plt.show()
